In [ ]:
import tensorflow as tf

from utils import *

## Load Data

In [ ]:
X_train, y_train = load_data(train_path)

n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

## Classification Models

### 1D-CNN

In [ ]:
input_ = tf.keras.layers.Input(shape=(n_timesteps, n_features))
norm = tf.keras.layers.experimental.preprocessing.Normalization()
norm.adapt(X_train)
# 1st convolutional block
conv1 = tf.keras.layers.Conv1D(filters=128, activation='relu', kernel_size=8, padding='same')(norm(input_))
conv1 = tf.keras.layers.BatchNormalization()(conv1)
# 2nd convolutional block
conv2 = tf.keras.layers.Conv1D(filters=256, activation='relu', kernel_size=5, padding='same')(conv1)
conv2 = tf.keras.layers.BatchNormalization()(conv2)
# 3rd convolutional block
conv3 = tf.keras.layers.Conv1D(filters=128, activation='relu', kernel_size=3, padding='same')(conv2)
conv3 = tf.keras.layers.BatchNormalization()(conv3)
# global average pooling
gap = tf.keras.layers.GlobalAveragePooling1D()(conv3)
# fully connected layer
output = tf.keras.layers.Dense(n_outputs, activation='softmax')(gap)

cnn1d_model = tf.keras.Model(inputs=[input_], outputs=[output], name='tmp_model')

cnn1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
history_1dcnn = cnn1d_model.fit(X_train, y_train, batch_size=64, epochs=200)

In [ ]:
# model_name = 'model_'
# cnn1d_model.save(model_path + model_name)

### Training acc and loss

In [ ]:
plt.figure(figsize=(18, 10))
plt.plot(history_1dcnn.history['categorical_accuracy'])
plt.xlabel('epoch')
plt.legend(['accuracy'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(18, 10))
plt.plot(history_1dcnn.history['loss'], color='orange')
plt.xlabel('epoch')
plt.legend(['loss'], loc='upper right')
plt.show()

### Confusion Matrix (1D-CNN)

In [ ]:
y_pred, y_true = get_predicted_values(cnn1d_model, test_path)
plot(y_pred, y_true)